In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
# import epicstore_api

In [5]:
gameurl = "https://www.steamgifts.com/discussion/S4e2c/free-epic-games-store-list-of-all-weekly-free-games-every-thursday-at-11-am-et"

r = requests.get(gameurl)

r.status_code

200

In [6]:
gsoup = BeautifulSoup(r.text)
gtable = gsoup.find("table")
gdf = pd.read_html(str(gtable))[0]
gdf.reset_index()
gdf  # gdf currently overwritten by other version below
# this above works but loses the EGS link which could be useful

,Number,Games,Start,End
0,477,The Bridge,3/14/2024,3/21/2024
1,476,Deus Ex: Mankind Divided,3/14/2024,3/21/2024
2,475,Astro Duel 2,3/7/2024,3/14/2024
3,474,Aerial_Knight’s Never Yield,2/29/2024,3/7/2024
4,473,Super Meat Boy Forever,2/22/2024,2/29/2024
...,...,...,...,...
472,5,Axiom Verge,2/7/2019,2/22/2019
473,4,The Jackbox Party Pack,1/24/2019,2/7/2019
474,3,What Remains of Edith Finch,1/11/2019,1/24/2019
475,2,Super Meat Boy,12/28/2018,1/10/2019


In [41]:
table_rows = gtable.find_all('tr')

result = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        result.append(row)
gdf = pd.DataFrame(result, columns=["Number", "Name", "Start", "End"])
print(gdf)
# above code adapter from https://stackoverflow.com/questions/50633050/scrape-tables-into-dataframe-with-beautifulsoup

    Number                         Name       Start         End
0      477                   The Bridge   3/14/2024   3/21/2024
1      476     Deus Ex: Mankind Divided   3/14/2024   3/21/2024
2      475                 Astro Duel 2    3/7/2024   3/14/2024
3      474  Aerial_Knight’s Never Yield   2/29/2024    3/7/2024
4      473       Super Meat Boy Forever   2/22/2024   2/29/2024
..     ...                          ...         ...         ...
472      5                  Axiom Verge    2/7/2019   2/22/2019
473      4       The Jackbox Party Pack   1/24/2019    2/7/2019
474      3  What Remains of Edith Finch   1/11/2019   1/24/2019
475      2               Super Meat Boy  12/28/2018   1/10/2019
476      1                   Subnautica  12/12/2018  12/27/2018

[477 rows x 4 columns]


In [51]:
# grabbing links to Epic Games Store pages separately from same scraped table
links = []
for tr in table_rows:
    td = tr.find_all('td')
    for cell in td:
        link = cell.find('a')
        if link:
            links.append(link['href'].strip())

links_series = pd.Series(links1)
links_series
# above chunk adapted from chatGPT corrections of own orginal code

gdf["Link"] = links_series
print(gdf)

0         https://store.epicgames.com/en-US/p/the-bridge
1      https://store.epicgames.com/en-US/p/deus-ex-ma...
2      https://store.epicgames.com/en-US/p/astro-duel...
3      https://store.epicgames.com/en-US/p/aerial-kni...
4      https://store.epicgames.com/en-US/p/super-meat...
                             ...                        
472      https://store.epicgames.com/en-US/p/axiom-verge
473    https://store.epicgames.com/en-US/p/jackbox-pa...
474    https://store.epicgames.com/en-US/p/what-remai...
475    https://store.epicgames.com/en-US/p/super-meat...
476       https://store.epicgames.com/en-US/p/subnautica
Length: 477, dtype: object

    Number                         Name       Start         End  \
0      477                   The Bridge   3/14/2024   3/21/2024   
1      476     Deus Ex: Mankind Divided   3/14/2024   3/21/2024   
2      475                 Astro Duel 2    3/7/2024   3/14/2024   
3      474  Aerial_Knight’s Never Yield   2/29/2024    3/7/2024   
4      473       Super Meat Boy Forever   2/22/2024   2/29/2024   
..     ...                          ...         ...         ...   
472      5                  Axiom Verge    2/7/2019   2/22/2019   
473      4       The Jackbox Party Pack   1/24/2019    2/7/2019   
474      3  What Remains of Edith Finch   1/11/2019   1/24/2019   
475      2               Super Meat Boy  12/28/2018   1/10/2019   
476      1                   Subnautica  12/12/2018  12/27/2018   

                                                  Link  
0       https://store.epicgames.com/en-US/p/the-bridge  
1    https://store.epicgames.com/en-US/p/deus-ex-ma...  
2    https://store.epicg

In [55]:
testurl = "https://store.epicgames.com/en-US/p/the-bridge"
rtest = requests.get(testurl)
rtest.status_code

403

In [4]:
import epicstore_api
from epicstore_api import EpicGamesStoreAPI, OfferData
import json

api = EpicGamesStoreAPI()

In [21]:
testgame = api.fetch_store_games(keywords = "The Bridge")
# testgame

title_to_find = "Blue Oak Bridge"
matching_item = None

for element in testgame["data"]["Catalog"]["searchStore"]["elements"]:
    if element["title"] == title_to_find:
        matching_item = element
        break

if matching_item:
    print("Title:", matching_item["title"])
    print("Description:", matching_item["description"])
    # You can extract other fields as needed
else:
    print("Title not found.")

# print(matching_item)
print(matching_item["price"]["totalPrice"]["originalPrice"])

# PRETTY JSON PRINTING
print(json.dumps(matching_item, indent=2, default=str))

Title: Blue Oak Bridge
Description: Blue Oak Bridge features breathtaking 2D hand-painted art in a storybook world of powerful witches, ancient magic and royal secrets. Farm, forage and fish throughout the island of Eloria and use all you discover to brew potions, make produce and help your new friends.
2499
{
  "title": "Blue Oak Bridge",
  "id": "aada95beab7c4671aa7cb3a6bb48c586",
  "namespace": "4dbe3446fbe1495a822cbea817f5606e",
  "description": "Blue Oak Bridge features breathtaking 2D hand-painted art in a storybook world of powerful witches, ancient magic and royal secrets. Farm, forage and fish throughout the island of Eloria and use all you discover to brew potions, make produce and help your new friends.",
  "effectiveDate": "2024-02-13T17:36:43.151Z",
  "keyImages": [
    {
      "type": "OfferImageWide",
      "url": "https://cdn1.epicgames.com/spt-assets/7faa14ae76ee4c15ae9d983ffc25224d/blue-oak-bridge-jd61u.jpg"
    },
    {
      "type": "OfferImageTall",
      "url": "h

In [104]:
def get_gamedata(gamename, tries = 10):
    # takes a string gamename, searches for it, finds exact match
    # returns dict with id, descr[iption], namespace, orig[inal]_price, fmt_orig_price (nicely formatted), and tags

    # MODIFIED to also grab seller name
    dict = api.fetch_store_games(count = tries, keywords = gamename)
    match = None
    for element in dict["data"]["Catalog"]["searchStore"]["elements"]:
        if element["title"] == gamename:
            match = element
            print("WE FOUND A MATCH")
            break
    to_return = {
    "id": match["id"],
    "descr": match["description"],
    "namespace": match["namespace"],
    "orig_price": match["price"]["totalPrice"]["originalPrice"],
    "fmt_orig_price": match["price"]["totalPrice"]["fmtPrice"]["originalPrice"],
    "tags": match["tags"],
    "seller": match["seller"]["name"],
    "items_id": match["items"][0]["id"],
    "items_namespace": match["items"][0]["namespace"],
    }
    return to_return
    # as a list (decided against):
    # return [match["id"], match["description"], match["namespace"], match["price"]["totalPrice"]["originalPrice"],
    #         match["price"]["totalPrice"]["fmtPrice"]["originalPrice"]]


jprint(get_gamedata("Blue Oak Bridge"))

WE FOUND A MATCH
{
  "id": "aada95beab7c4671aa7cb3a6bb48c586",
  "descr": "Blue Oak Bridge features breathtaking 2D hand-painted art in a storybook world of powerful witches, ancient magic and royal secrets. Farm, forage and fish throughout the island of Eloria and use all you discover to brew potions, make produce and help your new friends.",
  "namespace": "4dbe3446fbe1495a822cbea817f5606e",
  "orig_price": 2499,
  "fmt_orig_price": "$24.99",
  "tags": [
    {
      "id": "1393"
    },
    {
      "id": "1367"
    },
    {
      "id": "1370"
    },
    {
      "id": "9547"
    },
    {
      "id": "9549"
    },
    {
      "id": "1263"
    }
  ],
  "seller": "Lemon Seed Studio",
  "items_id": "50bd5de53e6e4863a383fd36bc855329",
  "items_namespace": "4dbe3446fbe1495a822cbea817f5606e"
}


In [6]:
blueoak = get_gamedata("Blue Oak Bridge")
blueoak["tags"]
# see url for tag readable names https://epicstore-api.readthedocs.io/en/latest/models.html

[{'id': '1393'},
 {'id': '1367'},
 {'id': '1370'},
 {'id': '9547'},
 {'id': '9549'},
 {'id': '1263'}]

In [42]:
# Create a quick function wrap that waits between each requests and backfills with blank data if failed
import time
def single_get(name):
    time.sleep(.5)
    try:    
        done = get_gamedata(name)
    except:
        try:
            done = get_gamedata(name, 100)
        except:
            done = None
        done = None
    return done

# The below code is what fetches all the API data for every listed game in free games
# WARNING TAKES A WHILE TO RUN!

# data_return_df = [single_get(a_game) for a_game in gdf['Name']]

In [95]:
# ff = gdf["Name"][3]
# get_gamedata("The Bridge")
# get_gamedata(ff)
# jprint(get_gamedata("Minit"))


{
  "id": "c3bb66a9e2064fa580597cee86cbc1df",
  "descr": "Minit is a peculiar little adventure played sixty seconds at a time.",
  "namespace": "43605e89e49d438485298819a7ef17b4",
  "orig_price": 999,
  "fmt_orig_price": "$9.99",
  "tags": [
    {
      "id": "1298"
    },
    {
      "id": "1370"
    },
    {
      "id": "9547"
    },
    {
      "id": "9548"
    },
    {
      "id": "1117"
    },
    {
      "id": "9549"
    },
    {
      "id": "10719"
    },
    {
      "id": "1263"
    }
  ],
  "seller": "Devolver Digital, Inc.",
  "items_id": "390e445663d14a2c893e736896977493",
  "items_namespace": "43605e89e49d438485298819a7ef17b4"
}


In [88]:
# test bench and helper test functions off of saved CSV data

# loads the csv into "dff"
dff = pd.read_csv("df_joined.csv")

def titleresults(title, trycounts = 10):
    # returns TITLES ONLY for a default 10 search
    resdict = api.fetch_store_games(count = trycounts, keywords = title)
    match_found = 0
    for element in resdict["data"]["Catalog"]["searchStore"]["elements"]:
        if element["title"] == title:
            print("\n\nMATCH FOUND!!!!\n\n")
            match_found = 1
        print(element["title"])
    return match_found

def getname(index_num):
    # grabs the full name from index to save me typing
    return (dff.iloc[index_num]["Name"])

def jprint(thing):
    print(json.dumps(thing, indent = 2, default = str))

def testsuite(index_num):
    name = getname(index_num)
    print("-> Testing", name)
    print("A search returns the following:", titleresults(name))
    if titleresults(name) == 1:
        jprint(get_gamedata(name))
    elif titleresults(name, 100) == 1:
        jprint(get_gamedata(name))
    else:
        print("\n\nsadface") # somethign was supposed to go here

In [109]:

    

# failure mode: Doors - Paradox != Doors: Paradox
# get_gamedata("Doors: Paradox")
# jprint(api.fetch_store_games(count = 10, keywords = "Doors: Paradox"))
# also deus ex: mankind same exact (space then dash)
# SOLUTION: add text replacer? Not sure if this happens ALWAYS though.

# failure case: Love
# TWO causes for failure: too many results (doesn't appear until 60, too general)
# titleresults("Love")
# SOLUTION: ??? Do it manually?
# listed in scrape as Love but LOVE is different caps.
# SOLUTION: add text lowercase coercion?
# jprint(api.fetch_store_games(count = 70, keywords = "Love"))
# jprint(get_gamedata("Love"))
# titleresults("Love", 30)
# get_gamedata("Love")

# strange case: Destiny 2: Legacy Collection
# doesn't return in search at all, but also, 

# The Outer Worlds: Spacer's Choice Edition
# DOES return on search but gamedata doesn't return anything

# print(getname(22))

# testsuite(22)
jprint(api.fetch_store_games(count = 1, keywords = getname(2)))
astro = api.fetch_store_games(count = 1, keywords = getname(2))
jprint(astro["id"])
spacer = api.fetch_store_games(count = 10, keywords = getname(22))
print(spacer["id"])

# get_gamedata(getname(2))
# get_gamedata(getname(22))

# getname(12)
# titleresults(getname(12))
# get_gamedata(getname(12)) # it DOES get found but cannot retrieve
# jprint(api.fetch_store_games(count = 5, keywords = getname(12)))
# jprint(api.fetch_store_games(count = 5, keywords = getname(2)))


{
  "data": {
    "Catalog": {
      "searchStore": {
        "elements": [
          {
            "title": "Astro Duel 2",
            "id": "bc38eac0277d41ec955690e25779cb53",
            "namespace": "4a681a809a094e2c8dcc68353c68fed6",
            "description": "Astro Duel 2 is a sci-fi combat game combining top-down space dog fights with up-close platforming action. Whether versus or co-op, battle through space and on foot at the same time! Strike within the fully-destructible arenas or just nuke \u2018em from orbit.",
            "effectiveDate": "2024-03-07T16:00:00.000Z",
            "keyImages": [
              {
                "type": "OfferImageWide",
                "url": "https://cdn1.epicgames.com/spt-assets/3251b927449a471c883aab00c076d63c/astro-duel-2-ukpi8.png"
              },
              {
                "type": "OfferImageTall",
                "url": "https://cdn1.epicgames.com/spt-assets/3251b927449a471c883aab00c076d63c/astro-duel-2-1w3a4.png"
              

KeyError: 'id'

In [27]:
dff.head(10)

,Unnamed: 0,id,descr,namespace,orig_price,fmt_orig_price,tags,Number,Name,Start,End,Link
0,0,6b228a1ad940496dabc89ec6640107f3,The Bridge,epic,999.0,$9.99,"[{'id': '1298'}, {'id': '1370'}, {'id': '9547'...",477,The Bridge,3/14/2024,3/21/2024,https://store.epicgames.com/en-US/p/the-bridge
1,1,NaN,NaN,NaN,NaN,NaN,NaN,476,Deus Ex: Mankind Divided,3/14/2024,3/21/2024,https://store.epicgames.com/en-US/p/deus-ex-ma...
2,2,bc38eac0277d41ec955690e25779cb53,Astro Duel 2 is a sci-fi combat game combining...,4a681a809a094e2c8dcc68353c68fed6,1999.0,$19.99,"[{'id': '1216'}, {'id': '21894'}, {'id': '1984...",475,Astro Duel 2,3/7/2024,3/14/2024,https://store.epicgames.com/en-US/p/astro-duel...
3,3,NaN,NaN,NaN,NaN,NaN,NaN,474,Aerial_Knight’s Never Yield,2/29/2024,3/7/2024,https://store.epicgames.com/en-US/p/aerial-kni...
4,4,ef66029a1c0d467ea7755f6a88b4088c,Super Meat Boy Forever,epic,1999.0,$19.99,"[{'id': '1216'}, {'id': '1370'}, {'id': '9547'...",473,Super Meat Boy Forever,2/22/2024,2/29/2024,https://store.epicgames.com/en-US/p/super-meat...
5,5,aca45f8a6b124590af039f97e148bd89,Dakar Desert Rally is the biggest off-road ral...,ac98aa34584648f2b71152e509191c1c,2999.0,$29.99,"[{'id': '1393'}, {'id': '21138'}, {'id': '1203...",472,Dakar Desert Rally,2/15/2024,2/22/2024,https://store.epicgames.com/en-US/p/dakar-dese...
6,6,186b5f89167f48fd94a9538ebfe214c6,Lost Castle is an action RPG beat'em up with r...,ab29925a0a9a49598adba45d108ceb3e,999.0,$9.99,"[{'id': '1264'}, {'id': '1265'}, {'id': '21894...",471,Lost Castle,2/8/2024,2/15/2024,https://store.epicgames.com/en-US/p/lost-castl...
7,7,76cd108ce4354545bd825e181262dbad,Welcome to the club! Write poems for your crus...,1311a5735ed844c0b3c47027cab321f0,1499.0,$14.99,"[{'id': '1218'}, {'id': '1367'}, {'id': '1370'...",470,Doki Doki Literature Club Plus!,2/8/2024,2/15/2024,https://store.epicgames.com/en-US/p/doki-doki-...
8,8,NaN,NaN,NaN,NaN,NaN,NaN,469,Doors: Paradox,2/1/2024,2/8/2024,https://store.epicgames.com/en-US/p/doors-para...
9,9,e69075bc6dd84fa09b47af91b3ce6723,Infinifactory is a sandbox puzzle game by Zach...,1260f09c5ab842b9a645e14f085ed5af,2499.0,$24.99,"[{'id': '1298'}, {'id': '21138'}, {'id': '1188...",468,Infinifactory,1/25/2024,2/1/2024,https://store.epicgames.com/en-US/p/infinifact...
